In [59]:
import pandas as pd
import numpy as np

# Load the CSV files into DataFrames
nba_17_18 = pd.read_csv("nba17-18.csv")
nba_18_19 = pd.read_csv("nba18-19.csv")
nba_19_20 = pd.read_csv("nba19-20.csv")
nba_20_21 = pd.read_csv("nba20-21.csv")
nba_21_22 = pd.read_csv("nba21-22.csv")
nba_22_23 = pd.read_csv("nba22-23.csv")
nba_23_24 = pd.read_csv("nba23-24.csv")

def clean_data(df):
    # Drop the first, third, and last two columns
    df = df.drop(df.columns[[0, 2, -1, -2]], axis=1)
    return df

# Clean each dataset
nba_17_18 = clean_data(nba_17_18)
nba_18_19 = clean_data(nba_18_19)
nba_19_20 = clean_data(nba_19_20)
nba_20_21 = clean_data(nba_20_21)
nba_21_22 = clean_data(nba_21_22)
nba_22_23 = clean_data(nba_22_23)
nba_23_24 = clean_data(nba_23_24)

# Function to filter rows where "Team" is '2TM' for each player
def filter_tm(df):
    # For each player, keep the row where "Team" is '2TM' or '3TM' if available, else keep the first row
    def get_tm_or_first(group):
        if "3TM" in group["Team"].values:
            return group[group["Team"] == "3TM"]
        elif "2TM" in group["Team"].values:
            return group[group["Team"] == "2TM"]
        else:
            return group.head(1)
    
    return df.groupby("Player", group_keys=False).apply(get_tm_or_first)

# Apply the filtering function to each dataset
nba_17_18 = filter_tm(nba_17_18)
nba_18_19 = filter_tm(nba_18_19)
nba_19_20 = filter_tm(nba_19_20)
nba_20_21 = filter_tm(nba_20_21)
nba_21_22 = filter_tm(nba_21_22)
nba_22_23 = filter_tm(nba_22_23)
nba_23_24 = filter_tm(nba_23_24)

# Add season identifiers to the column names (excluding the first column)
def add_season_prefix(df, season):
    df.columns = [df.columns[0]] + [f"{season}.{col}" for col in df.columns[1:]]
    return df

nba_17_18 = add_season_prefix(nba_17_18, "17-18")
nba_18_19 = add_season_prefix(nba_18_19, "18-19")
nba_19_20 = add_season_prefix(nba_19_20, "19-20")
nba_20_21 = add_season_prefix(nba_20_21, "20-21")
nba_21_22 = add_season_prefix(nba_21_22, "21-22")
nba_22_23 = add_season_prefix(nba_22_23, "22-23")
nba_23_24 = add_season_prefix(nba_23_24, "23-24")


In [60]:
nba_master_data = nba_17_18
for df in [nba_18_19, nba_19_20, nba_20_21, nba_21_22, nba_22_23, nba_23_24]:
    nba_master_data = pd.merge(nba_master_data, df, on="Player", how="outer")

In [61]:
nba_master_data[nba_master_data['Player']=='Obi Toppin']

,Player,17-18.Team,17-18.Pos,17-18.G,17-18.GS,17-18.MP,17-18.FG,17-18.FGA,17-18.FG%,17-18.3P,...,23-24.FT%,23-24.ORB,23-24.DRB,23-24.TRB,23-24.AST,23-24.STL,23-24.BLK,23-24.TOV,23-24.PF,23-24.PTS
847,Obi Toppin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.77,1.0,2.9,3.9,1.6,0.6,0.5,0.8,1.7,10.3


In [62]:
# filtering to first 3 nba seasons

# List of seasons in order
seasons = ['17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24']

# Function to get the first three seasons for each player
def get_first_three_seasons(row):
    # Find the first non-NaN season
    first_season_idx = None
    for season in seasons:
        if not pd.isna(row[f'{season}.G']):
            first_season_idx = seasons.index(season)
            break
    
    # If the player has less than 3 seasons, return NaN for the missing seasons
    if first_season_idx is None:
        return pd.Series([np.nan] * len(stats) * 3)
    
    # Extract the first three seasons
    yr1_season = seasons[first_season_idx]
    yr2_season = seasons[first_season_idx + 1] if first_season_idx + 1 < len(seasons) else None
    yr3_season = seasons[first_season_idx + 2] if first_season_idx + 2 < len(seasons) else None
    
    # Collect the stats for the first three seasons
    yr1_stats = row[[f'{yr1_season}.{stat}' for stat in stats]].values
    yr2_stats = row[[f'{yr2_season}.{stat}' for stat in stats]].values if yr2_season else [np.nan] * len(stats)
    yr3_stats = row[[f'{yr3_season}.{stat}' for stat in stats]].values if yr3_season else [np.nan] * len(stats)
    
    # Combine the stats into a single Series
    return pd.Series(np.concatenate([yr1_stats, yr2_stats, yr3_stats]))


In [63]:
# List of stats (excluding Player, Team, Pos, etc.)
stats = ['Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 
         'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Apply the function to each row
new_data = nba_master_data.apply(get_first_three_seasons, axis=1)

# Create new column names
new_columns = []
for yr in ['yr1', 'yr2', 'yr3']:
    for stat in stats:
        new_columns.append(f'{yr}.{stat}')

# Assign the new columns to the DataFrame
new_data.columns = new_columns


In [64]:
# Add the player names back to the new DataFrame
new_data['Player'] = nba_master_data['Player']

# Reorder columns to have 'Player' first
new_data = new_data[['Player'] + new_columns]

# Save the new DataFrame to a CSV file (optional)
new_data.to_csv('nba_three_szns.csv', index=False)

# Display the new DataFrame
print(new_data.head())

           Player yr1.Team yr1.Pos  yr1.G  yr1.GS  yr1.MP  yr1.FG  yr1.FGA  \
0    Aaron Brooks      MIN      PG   32.0     1.0     5.9     0.9      2.2   
1    Aaron Gordon      ORL      PF   58.0    57.0    32.9     6.5     14.9   
2  Aaron Harrison      DAL      SG    9.0     3.0    25.9     2.1      7.7   
3   Aaron Jackson      HOU      PG    1.0     0.0    35.0     3.0      9.0   
4     Abdel Nader      BOS      SF   48.0     1.0    10.9     1.0      3.1   

   yr1.FG%  yr1.3P  ...  yr3.FT%  yr3.ORB  yr3.DRB  yr3.TRB  yr3.AST  yr3.STL  \
0    0.406     0.3  ...      NaN      NaN      NaN      NaN      NaN      NaN   
1    0.434     2.0  ...    0.674      1.7      5.9      7.7      3.7      0.8   
2    0.275     1.0  ...      NaN      NaN      NaN      NaN      NaN      NaN   
3    0.333     1.0  ...      NaN      NaN      NaN      NaN      NaN      NaN   
4    0.336     0.5  ...    0.773      0.3      1.6      1.8      0.7      0.4   

   yr3.BLK  yr3.TOV  yr3.PF  yr3.PTS  
0    